<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#SpecD" data-toc-modified-id="SpecD-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>SpecD</a></span><ul class="toc-item"><li><span><a href="#Filter-by-sparsity" data-toc-modified-id="Filter-by-sparsity-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Filter by sparsity</a></span></li><li><span><a href="#Zero-estimation" data-toc-modified-id="Zero-estimation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Zero estimation</a></span></li><li><span><a href="#clr-transformation" data-toc-modified-id="clr-transformation-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>clr transformation</a></span></li><li><span><a href="#New-phyloseq-obj." data-toc-modified-id="New-phyloseq-obj.-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>New phyloseq obj.</a></span></li></ul></li><li><span><a href="#IndD" data-toc-modified-id="IndD-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>IndD</a></span><ul class="toc-item"><li><span><a href="#Filter-by-sparsity" data-toc-modified-id="Filter-by-sparsity-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Filter by sparsity</a></span></li><li><span><a href="#Zero-estimation" data-toc-modified-id="Zero-estimation-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Zero estimation</a></span></li><li><span><a href="#clr-transformation" data-toc-modified-id="clr-transformation-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>clr transformation</a></span></li><li><span><a href="#New-phyloseq-obj" data-toc-modified-id="New-phyloseq-obj-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>New phyloseq obj</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Create coda-transformed dataset
  * accounting for compositional data
* Method
  * impute zeros with zCompositions
  * perform clr transformation on counts
    * `compositions::clr`

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/'

# microbiome
## 16S-arch phyloseq object (one sample per species)
physeq_arch_SpecD_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_SpecD.RDS'
## 16S-arch phyloseq object (one sample per individual)
physeq_arch_IndD_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_IndD.RDS'

# params
threads = 8
my_seed = 8371


# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)

library(phyloseq)
library(zCompositions)

library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: NADA

Loading required package: survival


Attaching package: ‘NADA’


The following object is masked from ‘package:stats’:

    cor


Loading required package: truncnorm



In [3]:
df.dims()
set.seed(my_seed)
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/ 


# Load

In [4]:
# phyloseq
physeq_arch_SpecD = readRDS(physeq_arch_SpecD_file)
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1151 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 1151 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1151 tips and 1150 internal nodes ]

In [5]:
# phyloseq
physeq_arch_IndD = readRDS(physeq_arch_IndD_file)
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1891 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 1891 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1891 tips and 1888 internal nodes ]

# SpecD

## Filter by sparsity

In [6]:
# filter by sparsity
physeq_arch_SpecD = physeq_arch_SpecD %>%
    filter_taxa(function(x) sum(x > 0) >= 2, TRUE) %>%
    prune_samples(sample_sums(.) > 0, .)
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 225 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 225 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 225 tips and 224 internal nodes ]

In [7]:
# otu table
otu = physeq_arch_SpecD %>%
    phyloseq2df(otu_table) %>%
    as.data.frame
rownames(otu) = otu$OTU
otu$OTU = NULL
otu = otu %>% t %>% as.data.frame
otu

,6668cc03267d4f04b5a34d676c1db57a,cfd2bb81e97edddddf1a843a0d6999fa,51b922408976c428c8b582162eaefb1c,c0a04f628d6cf2335740fafaa08df6d9,617cfd32cb2af41ec4a5ba9ff3a05812,0bce7af71d4ef12c38f1d4550626d2ec,e3e571be3293bddcadd6be6a621d137b,669563302fb65e3b5f7097564fd8b880,376173310729d029029e14e72197f90d,7071f99f1374237a16119ec7838beea5,⋯,2de3f467031273dab285a370dc6fd85d,6c5b0990f1e56ac81d4f158ad3fe7ddf,f40a4552f05b5c3ca111c9886b7360a0,e50dfa4783c02bc4cbff9d9297e8d844,056a6332380b1f40d814d70adc074217,de8bd36d71e9372a6b9e16b54cefa562,890c6bb9fc8dea9dc4b756a645f4f472,b171d5cdecd235e7199d44ce8991794d,36fd5c55ad082f4d05194157da9d13da,d7dd0bdd8184f8cb7189e1e09afa97d0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0,0,0,0,0,0,0,48,0,0,⋯,0,0,0,0,0,0,0,0,0,0
F44_Fallow_Deer,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,0,0,0,0,0,0,0,0,22,0,⋯,0,0,0,0,0,0,0,0,0,0
X67_European_Roe,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


## Zero estimation

In [8]:
# sparsity (%)
otu %>%
    apply(2, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,52.72727,94.54545,97.27273,94.04444,98.18182,98.18182,17.938,7.323


In [9]:
# inferring zeros
## warning: a bit slow
otu = cmultRepl(otu, output = 'p-counts')
otu

No. corrected values:  1187 


,6668cc03267d4f04b5a34d676c1db57a,cfd2bb81e97edddddf1a843a0d6999fa,51b922408976c428c8b582162eaefb1c,c0a04f628d6cf2335740fafaa08df6d9,617cfd32cb2af41ec4a5ba9ff3a05812,0bce7af71d4ef12c38f1d4550626d2ec,e3e571be3293bddcadd6be6a621d137b,669563302fb65e3b5f7097564fd8b880,376173310729d029029e14e72197f90d,7071f99f1374237a16119ec7838beea5,⋯,2de3f467031273dab285a370dc6fd85d,6c5b0990f1e56ac81d4f158ad3fe7ddf,f40a4552f05b5c3ca111c9886b7360a0,e50dfa4783c02bc4cbff9d9297e8d844,056a6332380b1f40d814d70adc074217,de8bd36d71e9372a6b9e16b54cefa562,890c6bb9fc8dea9dc4b756a645f4f472,b171d5cdecd235e7199d44ce8991794d,36fd5c55ad082f4d05194157da9d13da,d7dd0bdd8184f8cb7189e1e09afa97d0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.1046535,0.06201692,0.2842442,0.1459982,0.1408301,0.5168077,0.4586668,48.0000000,0.2390235,0.2144752,⋯,0.07752115,0.03488452,2.311549,2.149920,0.9677223,0.0335925,1.414057,0.7050356,0.5155156,0.1369540
F44_Fallow_Deer,0.3256748,0.19299249,0.8845489,0.4543365,0.4382538,1.6082708,1.4273403,0.4422745,0.7438252,0.6674324,⋯,0.24124062,0.10855828,11.555426,6.690406,3.0114870,0.1045376,11.169441,6.6904065,1.6042501,0.4261918
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,0.08203047,0.04861065,0.2227988,0.1144376,0.1103867,0.4050888,0.3595163,0.1113994,22.0000000,0.1681118,⋯,0.06076331,0.02734349,1.729729,1.079079,0.7585287,0.02633077,1.058137,0.5275772,0.404076,0.1073485
X67_European_Roe,0.23311313,0.13814111,0.6331468,0.3252072,0.3136954,1.1511759,1.0216687,0.3165734,0.5324189,0.4777380,⋯,0.17267639,0.07770438,8.271199,4.788892,2.1555770,0.07482644,5.015845,2.5008525,1.148298,0.3050616


In [10]:
# check
## sparsity (%)
otu %>%
    apply(1, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,0,0,0,0,0,0,0,0


## clr transformation

In [11]:
# transforming
otu = compositions::clr(otu) %>%
    as.data.frame
otu

,6668cc03267d4f04b5a34d676c1db57a,cfd2bb81e97edddddf1a843a0d6999fa,51b922408976c428c8b582162eaefb1c,c0a04f628d6cf2335740fafaa08df6d9,617cfd32cb2af41ec4a5ba9ff3a05812,0bce7af71d4ef12c38f1d4550626d2ec,e3e571be3293bddcadd6be6a621d137b,669563302fb65e3b5f7097564fd8b880,376173310729d029029e14e72197f90d,7071f99f1374237a16119ec7838beea5,⋯,2de3f467031273dab285a370dc6fd85d,6c5b0990f1e56ac81d4f158ad3fe7ddf,f40a4552f05b5c3ca111c9886b7360a0,e50dfa4783c02bc4cbff9d9297e8d844,056a6332380b1f40d814d70adc074217,de8bd36d71e9372a6b9e16b54cefa562,890c6bb9fc8dea9dc4b756a645f4f472,b171d5cdecd235e7199d44ce8991794d,36fd5c55ad082f4d05194157da9d13da,d7dd0bdd8184f8cb7189e1e09afa97d0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,-1.101301,-1.624549,-0.1021229,-0.7683627,-0.8044026,0.49571407,0.3763673,5.027000,-0.2753946,-0.3837627,⋯,-1.401406,-2.199914,1.993716,1.921229,1.1229885,-2.237654,1.502262,0.8062916,0.49321094,-0.8323114
F44_Fallow_Deer,-1.676850,-2.200098,-0.6776718,-1.3439115,-1.3799514,-0.07983478,-0.1991815,-1.370819,-0.8509435,-0.9593115,⋯,-1.976955,-2.775462,1.892161,1.345680,0.5474397,-2.813203,1.858187,1.3456803,-0.08233791,-1.4078602
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,-1.107361,-1.630609,-0.10818212,-0.7744218,-0.8104618,0.4896549,0.3703081,-0.8013293,4.4843464,-0.3898219,⋯,-1.407465,-2.205973,1.941269,1.469412,1.116929,-2.243713,1.449814,0.753844,0.4871518,-0.8383706
X67_European_Roe,-1.083338,-1.606586,-0.08415915,-0.7503989,-0.7864388,0.5136778,0.3943311,-0.7773063,-0.2574309,-0.3657989,⋯,-1.383442,-2.181950,2.485673,1.939193,1.140952,-2.219690,1.985496,1.289526,0.5111747,-0.8143476


In [12]:
# summary
otu %>% rowSums %>% summary
otu %>% colSums %>% summary

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-9.337e-14 -1.230e-14 -8.674e-16 -1.402e-15  1.119e-14  1.574e-13 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -367.9  -122.3   -23.2     0.0   132.0   403.6 

## New phyloseq obj.

In [14]:
# new phyloseq object
physeq_arch_SpecD_clr = phyloseq(otu_table(otu %>% t %>% as.data.frame, 
                                           taxa_are_rows=TRUE),
                                tax_table(physeq_arch_SpecD),
                                sample_data(physeq_arch_SpecD),
                                phy_tree(physeq_arch_SpecD))
physeq_arch_SpecD_clr

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 225 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 225 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 225 tips and 224 internal nodes ]

In [15]:
# writing new file
physeq_arch_SpecD_clr_file = file.path(dirname(physeq_arch_SpecD_file),
                                      'physeq_arch_SpecD_clr.RDS')
saveRDS(physeq_arch_SpecD_clr, physeq_arch_SpecD_clr_file)
cat('File written:', physeq_arch_SpecD_clr_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_SpecD_clr.RDS 


# IndD

## Filter by sparsity

In [16]:
# filter by sparsity
physeq_arch_IndD = physeq_arch_IndD %>%
    filter_taxa(function(x) sum(x > 0) >= 2, TRUE) %>%
    prune_samples(sample_sums(.) > 0, .)
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 377 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 377 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 377 tips and 375 internal nodes ]

In [17]:
# otu table
otu = physeq_arch_IndD %>%
    phyloseq2df(otu_table) %>%
    as.data.frame
rownames(otu) = otu$OTU
otu$OTU = NULL
otu = otu %>% t %>% as.data.frame
otu

,c5fbed2bb10ab3a08275a92e9caea3c1,1936b1a270b058c5754be0addc7c7c31,cf3b465ae342367687f3fa74da2c46f0,6668cc03267d4f04b5a34d676c1db57a,6e9321684640a05dd16b7bc01b98c14a,0c59c44aeb355d3aa4dfa33ac4e77cde,2a64fe6cea748d320b2f99fe83449e46,458f7b9ac8ea99af3b251a761c878332,7950a70a07006f7c7b0c6f02c46a93af,9dd5def1c71e9aa144221be97b12d2e6,⋯,95cfc3c342d1f3a2af58c49f6065b08a,f40a4552f05b5c3ca111c9886b7360a0,e50dfa4783c02bc4cbff9d9297e8d844,056a6332380b1f40d814d70adc074217,de8bd36d71e9372a6b9e16b54cefa562,569a2abee10cb7fdef315334d856b92b,890c6bb9fc8dea9dc4b756a645f4f472,b171d5cdecd235e7199d44ce8991794d,36fd5c55ad082f4d05194157da9d13da,d7dd0bdd8184f8cb7189e1e09afa97d0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
F35_Red_Deer,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,61
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
X67_European_Roe,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


## Zero estimation

In [18]:
# sparsity (%)
otu %>%
    apply(1, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,86.47215,94.69496,96.28647,95.90508,97.61273,99.73475,4.571,1.866


In [19]:
# inferring zeros
## warning: a bit slow
otu = cmultRepl(otu, output = 'p-counts')
otu

No. corrected values:  4764 


,c5fbed2bb10ab3a08275a92e9caea3c1,1936b1a270b058c5754be0addc7c7c31,cf3b465ae342367687f3fa74da2c46f0,6668cc03267d4f04b5a34d676c1db57a,6e9321684640a05dd16b7bc01b98c14a,0c59c44aeb355d3aa4dfa33ac4e77cde,2a64fe6cea748d320b2f99fe83449e46,458f7b9ac8ea99af3b251a761c878332,7950a70a07006f7c7b0c6f02c46a93af,9dd5def1c71e9aa144221be97b12d2e6,⋯,95cfc3c342d1f3a2af58c49f6065b08a,f40a4552f05b5c3ca111c9886b7360a0,e50dfa4783c02bc4cbff9d9297e8d844,056a6332380b1f40d814d70adc074217,de8bd36d71e9372a6b9e16b54cefa562,569a2abee10cb7fdef315334d856b92b,890c6bb9fc8dea9dc4b756a645f4f472,b171d5cdecd235e7199d44ce8991794d,36fd5c55ad082f4d05194157da9d13da,d7dd0bdd8184f8cb7189e1e09afa97d0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.2695234,0.3277987,0.04127835,0.1966792,0.3909303,0.4831995,0.1602571,0.1675415,0.5414748,0.2258169,⋯,0.02670952,2.878422,12.11641,4.822283,0.09226926,0.08255670,9.13987,4.516788,1.146081,0.473487
F35_Red_Deer,0.2773891,0.3373651,0.04248301,0.2024191,0.4023391,0.4973011,0.1649340,0.1724310,0.5572771,0.2324071,⋯,0.02748901,3.366608,12.47001,4.963015,0.09496202,0.08496602,10.69001,5.282844,1.179528,61.000000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0.3108877,0.3781067,0.04761344,0.2268640,0.4509272,0.5573573,0.1848522,0.1932545,0.6245763,0.2604735,⋯,0.03080869,4.569213,13.975944,5.562370,0.10643003,0.09522687,14.508648,7.169959,1.3219731,0.5461541
X67_European_Roe,0.2029716,0.2468573,0.03108574,0.1481144,0.2944002,0.3638860,0.1206858,0.1261715,0.4077717,0.1700573,⋯,0.02011430,1.676119,4.732914,3.631546,0.06948577,0.06217148,5.322192,2.630148,0.8630864,0.3565717


## clr transformation

In [20]:
# transforming
otu = compositions::clr(t(otu)) %>%
    t %>% as.data.frame
otu

,c5fbed2bb10ab3a08275a92e9caea3c1,1936b1a270b058c5754be0addc7c7c31,cf3b465ae342367687f3fa74da2c46f0,6668cc03267d4f04b5a34d676c1db57a,6e9321684640a05dd16b7bc01b98c14a,0c59c44aeb355d3aa4dfa33ac4e77cde,2a64fe6cea748d320b2f99fe83449e46,458f7b9ac8ea99af3b251a761c878332,7950a70a07006f7c7b0c6f02c46a93af,9dd5def1c71e9aa144221be97b12d2e6,⋯,95cfc3c342d1f3a2af58c49f6065b08a,f40a4552f05b5c3ca111c9886b7360a0,e50dfa4783c02bc4cbff9d9297e8d844,056a6332380b1f40d814d70adc074217,de8bd36d71e9372a6b9e16b54cefa562,569a2abee10cb7fdef315334d856b92b,890c6bb9fc8dea9dc4b756a645f4f472,b171d5cdecd235e7199d44ce8991794d,36fd5c55ad082f4d05194157da9d13da,d7dd0bdd8184f8cb7189e1e09afa97d0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.1387614,0.1419004,0.139731,0.1458316,0.102588,0.1244376,0.1165754,0.1385505,0.1216122,0.1383319,⋯,0.1381729,-0.6199650,0.2210105,-0.07582190,0.1171848,0.1358529,-0.3086362,-0.2762924,0.1044886,0.09084924
F35_Red_Deer,0.1675274,0.1706665,0.168497,0.1745976,0.131354,0.1532036,0.1453414,0.1673166,0.1503782,0.1670979,⋯,0.1669390,-0.4633016,0.2497766,-0.04705585,0.1459509,0.1646190,-0.1519728,-0.1196291,0.1332547,4.94935398
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0.2815382,0.2846772,0.2825078,0.2886084,0.2453648,0.2672144,0.2593522,0.2813273,0.2643890,0.2811087,⋯,0.2809497,-0.1578663,0.3637873,0.0669549,0.2599617,0.2786297,0.1534625,0.1858063,0.2472654,0.2336260
X67_European_Roe,-0.1448277,-0.1416887,-0.1438581,-0.1377575,-0.1810011,-0.1591515,-0.1670137,-0.1450385,-0.1619769,-0.1452572,⋯,-0.1454161,-1.1607260,-0.7190091,-0.3594110,-0.1664042,-0.1477362,-0.8493972,-0.8170534,-0.1791004,-0.1927398


In [21]:
# summary
otu %>% rowSums %>% summary
otu %>% colSums %>% summary

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-1804.96  -104.70    60.98     0.00   151.16   271.71 

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-2.024e-13 -2.887e-14 -1.776e-15 -1.075e-16  1.808e-14  4.574e-13 

## New phyloseq obj

In [22]:
# new phyloseq object
physeq_arch_IndD_clr = phyloseq(otu_table(otu %>% t %>% as.data.frame, 
                                          taxa_are_rows=TRUE),
                                tax_table(physeq_arch_IndD),
                                sample_data(physeq_arch_IndD),
                                phy_tree(physeq_arch_IndD))
physeq_arch_IndD_clr

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 377 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 377 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 377 tips and 375 internal nodes ]

In [23]:
# writing new file
physeq_arch_IndD_clr_file = file.path(dirname(physeq_arch_IndD_file),
                                      'physeq_arch_IndD_clr.RDS')
saveRDS(physeq_arch_IndD_clr, physeq_arch_IndD_clr_file)
cat('File written:', physeq_arch_IndD_clr_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_IndD_clr.RDS 


# sessionInfo

In [24]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq-eco/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] LeyLabRMisc_0.1.6   zCompositions_1.3.4 truncnorm_1.0-8    
 [4] NADA_1.6-1.1        survival_3.1-8      MASS_7.3-51.5      
 [7] phyloseq_1.30.0     ggplot2_3.2.1       tidyr_1.0.0        
[10] dplyr_0.8.5        

loaded via a namespace (and not attached):
 [1] Biobase_2.